<a href="https://colab.research.google.com/github/harpreetvirkk/NLP-QuestionAnswerSystem/blob/main/UIP_QnA_BERTwSIF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
!git clone -l -s https://github.com/harpreetvirkk/NLP-QuestionAnswerSystem rep
%cd rep
!ls

fatal: destination path 'rep' already exists and is not an empty directory.
/content/rep
dataset  README.md  UIP_QnA.ipynb  UIP_QnA_SIF.ipynb


In [2]:
!pip3 install fse
!pip3 install transformers
!pip3 install torch

In [23]:
import unicodedata
import pandas as pd
import numpy as np
import re
import unicodedata
import fse
from fse import SplitIndexedList
import gensim.downloader as api
from fse.models import uSIF
from sklearn import metrics
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize  
import torch
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer, BertForQuestionAnswering
import matplotlib.pyplot as plt
import seaborn as sns
import logging
# logging.basicConfig(format='%(asctime)s : %(threadName)s : %(levelname)s : %(message)s', level=logging.INFO)
pd.set_option('display.max_colwidth', -1)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


In [4]:
df1 = pd.read_csv('./dataset/S08_question_answer_pairs.txt', sep='\t')
df2 = pd.read_csv('./dataset/S09_question_answer_pairs.txt', sep='\t')
df3 = pd.read_csv('./dataset/S10_question_answer_pairs.txt', sep='\t', encoding = 'ISO-8859-1')
frames = [df1, df2, df3]
df = pd.concat(frames)

In [5]:
def getArticleText(file):
  fpath = './dataset/text_data/'+file+'.txt.clean'
  try:
    f = open(fpath, 'r')
    text = f.read()
  except UnicodeDecodeError:
    f = open(fpath, 'r', encoding = 'ISO-8859-1')
    text = f.read()
  return text

In [6]:
df = df.dropna(subset=['ArticleFile'])
df['ArticleText'] = df['ArticleFile'].apply(lambda x: getArticleText(x))
df['ArticleText'] = df['ArticleText'].apply(lambda x: re.sub(r'(\n)+', '. ', x))
df = df.drop(['DifficultyFromQuestioner', 'DifficultyFromAnswerer', 'ArticleFile'], axis='columns')
df

,ArticleTitle,Question,Answer,ArticleText
0,Abraham_Lincoln,Was Abraham Lincoln the sixteenth President of...,yes,"Abraham Lincoln. Abraham Lincoln (February 12,..."
1,Abraham_Lincoln,Was Abraham Lincoln the sixteenth President of...,Yes.,"Abraham Lincoln. Abraham Lincoln (February 12,..."
2,Abraham_Lincoln,Did Lincoln sign the National Banking Act of 1...,yes,"Abraham Lincoln. Abraham Lincoln (February 12,..."
3,Abraham_Lincoln,Did Lincoln sign the National Banking Act of 1...,Yes.,"Abraham Lincoln. Abraham Lincoln (February 12,..."
4,Abraham_Lincoln,Did his mother die of pneumonia?,no,"Abraham Lincoln. Abraham Lincoln (February 12,..."
...,...,...,...,...
1453,Zebra,What areas do the Grevy's Zebras inhabit?,NaN,Zebra. Plains zebra. Grevy's zebra. Zebras are...
1454,Zebra,Which species of zebra is known as the common ...,"Plains Zebra (Equus quagga, formerly Equus bur...",Zebra. Plains zebra. Grevy's zebra. Zebras are...
1455,Zebra,Which species of zebra is known as the common ...,Plains Zebra,Zebra. Plains zebra. Grevy's zebra. Zebras are...
1456,Zebra,At what age can a zebra breed?,five or six,Zebra. Plains zebra. Grevy's zebra. Zebras are...


In [7]:
# stop_words = set(stopwords.words('english'))
def cleanQuestion(text):
  text = str(text)
  # wnl = nltk.stem.WordNetLemmatizer()
  text = text.lower()
  words = re.sub(r'[^\w\s]', '', text).split()
  # words = [word for word in words if not word in stop_words]
  return " ".join([word for word in words])

def cleanAnswer(text):
  text = str(text)
  # wnl = nltk.stem.WordNetLemmatizer()
  text = text.lower()
  words = re.sub(r'[^\w\s]', '', text).split()
  # words = [word for word in words if not word in stop_words]
  return " ".join([word for word in words])

def cleanText(text):
  text = str(text)
  # wnl = nltk.stem.WordNetLemmatizer()
  text = text.lower()
  words = re.sub(r'[^\w\s\.\?]', '', text).split()
  # words = [word for word in words if not word in stop_words]
  return " ".join([word for word in words])

In [8]:
df['Question'] = df['Question'].apply(lambda x: cleanQuestion(x))
df['Answer'] = df['Answer'].apply(lambda x: cleanAnswer(x))
df['ArticleText'] = df['ArticleText'].apply(lambda x: cleanText(x))
df

,ArticleTitle,Question,Answer,ArticleText
0,Abraham_Lincoln,was abraham lincoln the sixteenth president of...,yes,abraham lincoln. abraham lincoln february 12 1...
1,Abraham_Lincoln,was abraham lincoln the sixteenth president of...,yes,abraham lincoln. abraham lincoln february 12 1...
2,Abraham_Lincoln,did lincoln sign the national banking act of 1863,yes,abraham lincoln. abraham lincoln february 12 1...
3,Abraham_Lincoln,did lincoln sign the national banking act of 1863,yes,abraham lincoln. abraham lincoln february 12 1...
4,Abraham_Lincoln,did his mother die of pneumonia,no,abraham lincoln. abraham lincoln february 12 1...
...,...,...,...,...
1453,Zebra,what areas do the grevys zebras inhabit,nan,zebra. plains zebra. grevys zebra. zebras are ...
1454,Zebra,which species of zebra is known as the common ...,plains zebra equus quagga formerly equus burch...,zebra. plains zebra. grevys zebra. zebras are ...
1455,Zebra,which species of zebra is known as the common ...,plains zebra,zebra. plains zebra. grevys zebra. zebras are ...
1456,Zebra,at what age can a zebra breed,five or six,zebra. plains zebra. grevys zebra. zebras are ...


In [9]:
use_cuda = True

In [10]:
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [11]:
def getGraph(start_scores, end_scores, tokens):

  # x-axis: unique tokens
  # y-axis: start word score for each token

  # Pull the scores from Tensors and convert to 1-D np arrays
  start_scores = start_scores.detach().numpy().flatten()
  end_scores = end_scores.detach().numpy().flatten()

  # We add unique token index to each token
  token_label = list()
  for (i, t) in enumerate(tokens):
      token_label.append('{:} - {:>2}'.format(t, i))

  # Plot Graph
  sns.set(style='darkgrid')
  plt.rcParams["figure.figsize"] = (24,8)
  ax = sns.barplot(x=token_labels, y=s_scores, ci=None)
  ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha="center")
  ax.grid(True)
  plt.title('Start Word Scores for Each unique Token in Context')
  plt.show()

In [12]:
def getSim(q, x):
  x = (str(x).split(), 0)
  sim = metrics.pairwise.cosine_similarity(sifmodel.infer([q]), sifmodel.infer([x]))
  return sim

def getBestAnswer(question, potentials):
  q = (str(question).split(), 0)
  c = pd.DataFrame(potentials)
  c['sim'] = c[0].apply(lambda x: getSim(q, x))
  max = c.sort_values(by='sim', ascending=False).iloc[:3]
  return max[0]

In [13]:
from fse.models import uSIF
glove = api.load("glove-wiki-gigaword-100")
sifmodel = uSIF(glove, workers=2, lang_freq="en")

In [14]:
sifmodel.train(SplitIndexedList(list(df['ArticleText'])))

(3996, 17283272)

In [15]:
def get_split(text1):

  #Reference: https://medium.com/@armandj.olivares/using-bert-for-classifying-documents-with-long-texts-5c3e7b04573d
  
  l_total = []
  l_parcial = []
  if len(text1.split())//250 >0:
    n = len(text1.split())//250
  else: 
    n = 1
  for w in range(n):
    if w == 0:
      l_parcial = text1.split()[:350]
      l_total.append(" ".join(l_parcial))
    else:
      l_parcial = text1.split()[w*250:w*250 + 350]
      l_total.append(" ".join(l_parcial))
  return l_total

In [24]:
def getAnswer(question, context):

  # print('Query Context has {} tokens.'.format(len(tokenizer.encode(context))))

  context_list = get_split(context)

  ans = []

  for c in context_list:

    encoding = tokenizer.encode_plus(text=question,text_pair=c)

    inputs = encoding['input_ids']  #Token embeddings
    sentence_embedding = encoding['token_type_ids']  #Segment embeddings
    tokens = tokenizer.convert_ids_to_tokens(inputs) #input tokens

    start_scores, end_scores = model(input_ids=torch.tensor([inputs]), token_type_ids=torch.tensor([sentence_embedding]))

    start_index = torch.argmax(start_scores)

    end_index = torch.argmax(end_scores)

    answer = ' '.join(tokens[start_index:end_index+1])

    ans.append(answer)
  print('Question: ', question)

  potentials = []
  for i in ans:
    if ('SEP' not in i) and ('CLS' not in i):
      potentials.append(re.sub('(#)+', '', i))

  answer = getBestAnswer(question, potentials)

  print('Potential Answers: \n')
  print(answer.head())

In [25]:
getAnswer(df['Question'].iloc[17], df['ArticleText'].iloc[17])
print('Actual Answer: ', df['Answer'].iloc[17])

Question:  did lincoln beat john c breckinridge in the 1860 election
Potential Answers: 

0    lincoln won the republican party nomination in 1860 and was elected president later that year
1    lincoln was elected as the 16th president of the united states                               
3    he easily defeated efforts to deny his reno mina tion                                        
Name: 0, dtype: object
Actual Answer:  yes


In [26]:
getAnswer(df['Question'].iloc[388], df['ArticleText'].iloc[388])
print('Actual Answer: ', df['Answer'].iloc[388])

Question:  what allows a duck to filter water out of the side of their beaks and keep food inside
Potential Answers: 

1    tiny rows of plates called lame lla e
0    lame lla e                           
Name: 0, dtype: object
Actual Answer:  tiny rows of plates called lamellae like a whales baleen


In [27]:
getAnswer(df['Question'].iloc[300], df['ArticleText'].iloc[300])
print('Actual Answer: ', df['Answer'].iloc[300])

Question:  which countries established colonies in canada
Potential Answers: 

1    britain and the united states
2    british and french colonies  
0    british and french           
Name: 0, dtype: object
Actual Answer:  nan


In [20]:
getAnswer(df['Question'].iloc[255], df['ArticleText'].iloc[255])
print('Actual Answer: ', df['Answer'].iloc[255])

Question:  what years were coolidges two sons born in
Potential Answers: 
 2    1906
0    1872
1    1872
Name: 0, dtype: object
Actual Answer:  1906 and 1908


In [28]:
getAnswer(df['Question'].iloc[171], df['ArticleText'].iloc[171])
print('Actual Answer: ', df['Answer'].iloc[171])

Question:  what do beetles eat
Potential Answers: 

0    they often feed on plants and fungi break down animal and plant debris and eat other invertebrates
1                                                                                                      
Name: 0, dtype: object
Actual Answer:  they often feed on plants and fungi break down animal and plant debris and eat other invertebrates


In [29]:
getAnswer(df['Question'].iloc[666], df['ArticleText'].iloc[666])
print('Actual Answer: ', df['Answer'].iloc[666])

Question:  do wolves leave their pack
Potential Answers: 

1    the instinct to reproduce drives young wolves away from their birth packs leading them to seek out mates and territories of their own
2    the instinct to reproduce drives young wolves away from their birth packs leading them to seek out mates and territories of their own
3    many of them will be compelled to leave their birth packs and seek out mates and territories of their own                            
Name: 0, dtype: object
Actual Answer:  they leave their birth pack when they reach sexual maturity


In [32]:
getAnswer(df['Question'].iloc[999], df['ArticleText'].iloc[999])
print('Actual Answer: ', df['Answer'].iloc[999])

Question:  what is a hybrid animal resulting from a union between a leopard and a puma
Potential Answers: 

2    leopard ess
0    leopard    
1    pu ma par d
Name: 0, dtype: object
Actual Answer:  a pumapard


In [33]:
getAnswer(df['Question'].iloc[113], df['ArticleText'].iloc[113])
print('Actual Answer: ', df['Answer'].iloc[113])

Question:  he is what
Potential Answers: 

1    a great italian chemist
0    an italian sava nt     
Name: 0, dtype: object
Actual Answer:  nan


In [34]:
getAnswer(df['Question'].iloc[1166], df['ArticleText'].iloc[1166])
print('Actual Answer: ', df['Answer'].iloc[1166])

Question:  how many species of otter are there
Potential Answers: 

0    13 species
Name: 0, dtype: object
Actual Answer:  13
